In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import random
import pymongo
from tenacity import retry, wait_fixed, stop_after_attempt, retry_if_exception_type
import time
import requests
import json
import time
import hashlib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import gridfs
import base64
from io import BytesIO 
from PIL import Image
import matplotlib.pyplot as plt
from urllib.parse import urlparse, parse_qs
import hashlib
from bson import ObjectId
from datetime import datetime



In [6]:
# Setup Firefox options to connect to the existing session
firefox_options = Options()
firefox_options.set_preference("devtools.debugger.remote-enabled", True)
firefox_options.set_preference("devtools.debugger.remote-port", 9228)
firefox_options.add_argument("-profile") 
firefox_options.add_argument(r"C:\Users\Alime\AppData\Roaming\Mozilla\Firefox\Profiles\rzhn0kb7.mobile")

# Specify the path to your Firefox executable
firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox/firefox.exe'

# Path to your GeckoDriver
driver_path = r'C:\Users\Alime\OneDrive\Desktop\ig_scrape\geckodriver.exe'

service = Service(executable_path=driver_path)

database_name = 'Divar_Crawler'
collection_name = 'metadata'

In [7]:
driver = webdriver.Firefox(service=service, options=firefox_options)

time.sleep(3)
# URL of the Instagram post
post_url = 'https://divar.ir/s/tehran/mobile-phones'

# Navigate to the post URL
driver.get(post_url)

# UTILITY FUNCTIONS 1

In [8]:
# Custom JSON encoder to handle ObjectId
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super(JSONEncoder, self).default(o)
def encrypt_string(hash_string):
    sha_signature = \
        hashlib.sha256(hash_string.encode()).hexdigest()
    return sha_signature

def filter_invalid_values(data):
    return {key: value for key, value in data.items() if value not in (None, "")}

def is_base64_image(data_uri): 
    return data_uri.startswith('data:image/')

def decode_base64_image(base64_string):
    header, encoded = base64_string.split(",", 1) if "," in base64_string else ("", base64_string)
    image_data = base64.b64decode(encoded)
    image = Image.open(BytesIO(image_data))
    if image.mode != 'RGB':
        image = image.convert('RGB')
    if image.size != (200, 200): 
        image = image.resize((200, 200))
    return image

# Helper function to find the element or return None
def get_text_or_none(element):
    try:
        return element.find_element(By.CLASS_NAME, "kt-post-card__red-text").text
    
    except NoSuchElementException:
        return None
def get_status_or_none(element):
    try: 
        return element.find_element(By.CLASS_NAME,"kt-post-card__description").text
    except NoSuchElementException:
        return None

def get_price_or_none(element):
    try: 
        s = element.find_elements(By.CLASS_NAME,"kt-post-card__description")
        try:
            return s[1].text
        except:
            return None
    except NoSuchElementException:
        return None

    
def get_amn_or_none(element):
    try:
        s= element.find_elements(By.CSS_SELECTOR, ".kt-tag__text.kt-text-truncate")
        string = ""
        if len(s)<=0:
            return None
        else:
            for item in s:
                string += item.text
                string +="@"
            return string
    except NoSuchElementException:
        return None
    
def get_images_or_none(element):
    try:
        s = element.find_element(By.TAG_NAME,"img").get_attribute('src')
        return s
    except:
        return None 
    
def jsonize_data(items,links,images,titles,status,prices,time_string,nardebans,amns):
    # Creating the list of JSON objects
    json_data = []

    for i in range(len(items)):
        if images[i]:
            card_data = {
                "link": links[i],
                "image": images[i],
                "title": titles[i],
                "status": status[i],
                "price": prices[i],
                "time_string": time_string[i]
            }
        else:
            card_data = {
                "link": links[i],
                "title": titles[i],
                "status": status[i],
                "price": prices[i],
                "time_string": time_string[i]
            }
        
        card_data = filter_invalid_values(card_data)
        # Add optional fields if they are not None
        other_data = {}
        if nardebans[i] is not None:
            other_data["nardeban"] = nardebans[i]
        if amns[i] is not None:
            other_data["amn"] = amns[i]
        if other_data: 
            card_data["other"] = other_data
        json_data.append({"category": "موبایل","card_data": card_data})
    
    return json_data

def insert_data(json_data):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")  # Adjust the connection string as needed
    db = client[database_name]
    fs = gridfs.GridFS(db)
    metadata_collection = db[collection_name]

    
    for item in json_data:
        metadata = item["card_data"]

        founded_id = encrypt_string(metadata["title"]+metadata["link"])
        if (db[collection_name].find_one({"_id":founded_id})):
            pass
        else:
            try:
                image_url = metadata["image"]
                if is_base64_image(image_url): 
                    pass
                else:
                    response = requests.get(image_url)
                    if response.status_code == 200:
                        image_content = response.content
                    
                        grid_fs_id = fs.put(image_content, filename=founded_id+".jpg")

                        # Store the metadata along with the GridFS file ID
                        metadata["image_id"] = grid_fs_id
                    else:
                        print(f"Failed to download the image {metadata['title']}. Status code: {response.status_code}")
            except: 
                    pass
            
            page_data = scrape_page(metadata["link"])
            # try:
            #     page_data = scrape_page(metadata["link"])
            # except:
            #     driver.close()
            #     driver.switch_to.window(driver.window_handles[0])
            #     try: 
            #         page_text = driver.find_element(By.CSS_SELECTOR,"body > main > h1").text
            #         print("HOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO")
            #         if (page_text=="429"):
            #             print("429 ERROR------------------------------------")
            #             # give it another chance
            #             page_data = scrape_page(metadata["link"])
            #         else:
            #             pass
            #     except:
            #         pass
            print("page scraped")
            timestamp = datetime.now()
            metadata_collection.insert_one({"_id":founded_id,"category": "موبایل","card_data": metadata,"page_data":page_data,"timestamp":timestamp})
            print("data inserted")


        


# UTILITY FUNCTIONS 2

In [9]:
def is_base64_image(src):
    return src.startswith('data:image')

def hash_bytes(data):
    if not isinstance(data, bytes):
        raise TypeError("Input must be of type 'bytes'")
    
    # Create a new hashlib object
    hash_object = hashlib.sha256()
    
    # Update the hash object with the bytes data
    hash_object.update(data)
    
    # Get the hexadecimal representation of the hash
    hash_hex = hash_object.hexdigest()
    
    return hash_hex

def has_image_dbbad_class(driver):
    try:
        # Find the element using Selenium
        driver.find_element(By.CLASS_NAME, 'image-dbbad')
        return True
    except:
        return False
    
def extract_coordinates(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)
    
    # Get the latitude and longitude
    latitude = query_params.get('latitude', [None])[0]
    longitude = query_params.get('longitude', [None])[0]
    
    # Convert to float if they exist
    if latitude and longitude:
        latitude = float(latitude)
        longitude = float(longitude)
    
    return latitude, longitude


def get_description(element):
    #section.post-page__section--padded:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)
    #section.post-page__section--padded:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) 
    #section.post-page__section--padded:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)
    try:
        descs = element.find_element(By.CSS_SELECTOR, 'section.post-page__section--padded:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)')
        if not descs:
            print("Description element not found")
            return None
        return descs.text
    except AttributeError as e:
        print(f"Problem with description: {e}")
        return None

            
    except AttributeError as e:
        print(f"Problem with description: {e}")
        return None

def get_banner(element):
    pass



def get_grid_attribute(element):
    try:
        attribute = element.find_element(By.CSS_SELECTOR, '.kt-base-row__end.kt-unexpandable-row__value-box')
        print(attribute.text)
        if not attribute:
            print("grid_attribute not found")
            return None
        return attribute.text
    except AttributeError as e:
        print(f"Problem with grid_attribute: {e}")
        return None
    
def get_grid_logic(element,attributes_list):
    try:
                        
            title = element.find_element(By.CSS_SELECTOR, '.kt-base-row__start.kt-unexpandable-row__title-box').text
            attribute = get_grid_attribute(element)
            if(title == "برند و مدل"):
                attributes_list[0] = attribute
            elif(title == "وضعیت"):
                pass
            elif(title == "اصالت برند"):
                attributes_list[1] = attribute
            elif(title == "تعداد سیم‌کارت"):
                attributes_list[2] = attribute
            elif(title == "حافظهٔ داخلی"):
                attributes_list[3] = attribute
            elif(title == "مقدار رم"):
                attributes_list[4] = attribute
            elif(title == "رنگ"):
                attributes_list[5] = attribute
            elif(title == "مایل به معاوضه"):
                attributes_list[6] = attribute
            elif(title == "قیمت"):
                pass

    except:
            pass
    
    return attributes_list


def get_grid(element):

    attributes_list = ["","","","","","",""]
    try:
        grid = element.find_elements(By.CSS_SELECTOR, '.kt-base-row.kt-base-row--large.kt-unexpandable-row')
        
        for row in grid:
            try:
                attributes_list = get_grid_logic(row,attributes_list)
            except:
                pass

        return attributes_list
    except:
        return attributes_list


    
def get_the_tag(element):
    try:
        tags = element.find_elements(By.CSS_SELECTOR,'.kt-chip.kt-chip--has-action.kt-wrapper-row__child')
        if not tags:
            print("tag element not found")
            return None
        output = []
        for tag in tags:
            output.append(tag.text)

        return output
    except AttributeError as e:
        print(f"Problem with tag: {e}")
        return None

def get_images(driver):
    try:
        # Find all image elements
        images = driver.find_elements(By.TAG_NAME, 'img')
        
        # Extract 'src' attributes, excluding base64 images
        src_list = [
                    img.get_attribute('src') for img in images 
                    if img.get_attribute('src') and not is_base64_image(img.get_attribute('src')) and not contains_keyword(img.get_attribute('src'))
                ]        
        # Check if the class 'image-dbbad' exists and remove the last element if it does
        if has_image_dbbad_class(driver) and src_list:
            src_list.pop()
            
        print(has_image_dbbad_class(driver))
        print(src_list)
        return src_list
    except Exception as e:
        print(f"Problem with images: {e}")
        return None

def get_geopoints(elemnet):
    try:
        if has_image_dbbad_class(elemnet)==True:
            picture_element = driver.find_element(By.CLASS_NAME, "image-dbbad")
            picture_element.click()
            str = elemnet.find_element(By.CLASS_NAME,'map-cm__attribution.map-cm__button').get_attribute('href')
            return extract_coordinates(str)
        else: 
            return ""
    except:
        return ""
    



def contains_keyword(url):
    keywords = ["thumbnail", "open-platform","widget-icons","business_marketplace"]
    for keyword in keywords:
        if keyword in url:
            return True
    return False

# CORE 2

In [10]:
# Function to scrape a page
def scrape_page(link):
   
        # Connect to MongoDB
        client = pymongo.MongoClient("mongodb://localhost:27017/")  # Adjust the connection string as needed
        db = client[database_name]
        fs = gridfs.GridFS(db)
        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])
        
        # Perform your scraping logic here
        driver.get(link)
        time.sleep(2)  # Wait for the page to load
        
        attributes_list = get_grid(driver)

        page_data = {
            "brand_model":attributes_list[0],
            "brand_origin":attributes_list[1],
            "sim_count":attributes_list[2],
            "internal_storage":attributes_list[3],
            "ram":attributes_list[4],
            "color": attributes_list[5],
            "subsitute":attributes_list[6],
            "description": get_description(driver),
            "tags": get_the_tag(driver)
        }
        
        page_data = filter_invalid_values(page_data)
        images_url = get_images(driver)
        images = []
        try:
            for image_url in images_url:
                response = requests.get(image_url)
                if response.status_code == 200:
                    image_content = response.content
                    grid_fs_id = fs.put(image_content, filename=hash_bytes(image_content)+".jpg")
                # Store the metadata along with the GridFS file ID
                    images.append(grid_fs_id)
                    page_data["images_id"] = images
                else:
                    print("prob!!!!")
                
        except: 
            pass
        

        geo_points = get_geopoints(driver)
        if geo_points:
            page_data["geo_point"] = {"long":geo_points[0],"lat":geo_points[1]}
        print(page_data)
        driver.close()  # Close the current tab
        
        if driver.window_handles:  # Check if there are any tabs left
            driver.switch_to.window(driver.window_handles[0])  # Move to the next tab (last opened tab)

        return page_data


            

# CORE 1

In [11]:
def run():    
    items = []
    conn = pymongo.MongoClient("mongodb://localhost:27017/")  # Adjust the connection string as needed
    db = conn[database_name]
    try:
        #--------------
        items = driver.find_elements(By.CLASS_NAME,"widget-col-d2306")

        links = [element.find_element(By.TAG_NAME,"a").get_attribute('href') for element in items]
        images = list(map(get_images_or_none, items))
        titles= [element.find_element(By.CLASS_NAME,"kt-post-card__title").text for element in items]
        status = list(map(get_status_or_none, items))
        prices = list(map(get_price_or_none, items))
        time_string = [element.find_element(By.CSS_SELECTOR,".kt-post-card__bottom-description.kt-text-truncate").text for element in items]
        nardebans= list(map(get_text_or_none, items))
        amns= list(map(get_amn_or_none, items))
        data_in_json =  jsonize_data(items,links,images,titles,status,prices,time_string,nardebans,amns)
        insert_data(data_in_json)
        
        while True:

            # Check if the "load more" button is present
            try:

                load_more_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div[2]/div[2]/div/button')  # Adjust the XPath according to the button text
                if load_more_button:
                    load_more_button.click()
                    print("button cliked")
                    time.sleep(5)
                
            # Scroll down the page                
            except:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(5)


             
            print(db[collection_name].count_documents({}))
        #--------------
            items = driver.find_elements(By.CLASS_NAME,"widget-col-d2306")

            links = [element.find_element(By.TAG_NAME,"a").get_attribute('href') for element in items]
            images = list(map(get_images_or_none, items))
            titles= [element.find_element(By.CLASS_NAME,"kt-post-card__title").text for element in items]
            status = list(map(get_status_or_none, items))
            prices = list(map(get_price_or_none, items))
            time_string = [element.find_element(By.CSS_SELECTOR,".kt-post-card__bottom-description.kt-text-truncate").text for element in items]
            nardebans= list(map(get_text_or_none, items))
            amns= list(map(get_amn_or_none, items))
            data_in_json =  jsonize_data(items,links,images,titles,status,prices,time_string,nardebans,amns)
            insert_data(data_in_json)



                
                

    except Exception as e:
        print(f"An error occurred: {e}")



In [ ]:
while(True):
    run()
    print('---start----')
    num_tabs = len(driver.window_handles)
    if num_tabs>=2:  # Check if there are any tabs left
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        print('HASSSSSS ERRRRRROOOOOOOOOOOOOR')
    try:
        load_more_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/main/div[2]/div[2]/div/button')  # Adjust the XPath according to the button text
        if load_more_button:
            load_more_button.click()
            print("button cliked")
            time.sleep(5)
        try_again_button = driver.find_element(By.CSS_SELECTOR,'.kt-button.kt-button--primary.kt-button--outlined.error-message__button-c35bb')
        if try_again_button:
            try_again_button.click()
            print("button cliked")
            time.sleep(5)
    # Scroll down the page                
    except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    print('----end----')

# EXTRA

In [107]:
import pymongo
import gridfs
import os
import re

# Directory to save images
output_dir = "FOO1"
os.makedirs(output_dir, exist_ok=True)

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  # Adjust the connection string as needed
db = client[database_name]
fs = gridfs.GridFS(db)

# Query GridFS and save images to the local folder
i=1
for grid_out in fs.find({}):
    # Sanitize the filen ame and ensure each file is saved with a .jpg extension
    filename = str(i)+ grid_out.filename
    i+=1
    file_data = grid_out.read()
    
    with open(os.path.join(output_dir, filename), 'wb') as file:
        file.write(file_data)
        print(f"Saved: {filename}")

print("Images exported successfully.")

Saved: 1f0ace8d7d9d9967374e0b062472d67ec517030d641b4f5bb1b4b9b1168ec9075.jpg
Saved: 26416e35406a9bbc37aa65cf9cd149947c07d2302441438a33cbb34d207471dba.jpg
Saved: 3168d48646d01950fcdfd48cf12060e7d8e52bb5dec60224dcc486c52eafa76c0.jpg
Saved: 47111659a823fc015885548b95a7f228e0472289ed8a5a8d3b8a48ab924237f98.jpg
Saved: 5d80b17da9d18320ab84f50933324f5d794bde6b7205c7d830be9709417ad80af.jpg
Saved: 6fdb161660e280a46c9986c759581e3ae81f1ecaae5bd8aebe46d736b9b136248.jpg
Saved: 73ce9a310a7f384a0a36a16d0eddd46708683e7cf2c16af03f59d51b10cb4858d.jpg
Saved: 8602ec0c9ffeaa2208952583dbe1fead7bc7b9b1735b6992d2b10a54acea3bb9f.jpg
Saved: 9450e509f4fff96dc2177baa0d5a003cec3ef212d7cfb63d498bc92fdcdd73330.jpg
Saved: 102e5b95469296b656ae1495fc9aa5bc53aaf3f7c7298d610a44a83ac51165c302.jpg
Saved: 11915cd82cf6eb0dec78408856517866075991a74745e7ded63260cff456b11f97.jpg
Saved: 12272482cc8917e43b27e8c39dbec8736e98852888519e514c9f75a4d810989655.jpg
Saved: 13c4ed1881c50992646dfae58b7f849e5d44d3c9e0f0e1ba142db993a41870a63c

In [ ]:
driver.switch_to.window(driver.window_handles[0])  # Move to the next tab (last opened tab)

In [ ]:
driver.switch_to.window(driver.window_handles[0])  # Move to the next tab (last opened tab)